# Train example

Use this notebook to train the model.

In [ ]:
# Notebook setup

import os
REPO_ROOT = "D:\\play\\kelly-criterion"
os.chdir(REPO_ROOT)
print(f"Working directory: {os.getcwd()}")

In [ ]:
# Imports and function definitions

import matplotlib.pyplot as plt
import numpy as np

from src.environment import Environment
from src.agent import Agent

def run_training(train_from_fresh, num_episodes, epsilons, exploring_start=False):
    env = Environment()

    if train_from_fresh:
        agent = Agent(env)

        total_rewards = agent.train(env, num_episodes=num_episodes, epsilons_each_episode=epsilons,
                                    exploring_start=exploring_start, plot_training_rewards=True,
                                   use_tensorboard=True)

        agent.plot_N_values()

    else:
        agent = Agent(Q_values=np.loadtxt("Q.csv"))

    agent.plot_Q_values()

    agent.plot_policy(
        optimal_policy=[min(250 - s, 0.2 * s) for s in range(1, 250)]
    )


def epsilon_picket_fence(num_episodes: int, explore_ratio: float, repeats: int):
    repeating_phase_length = num_episodes // repeats
    single_explore_phase_length = int(repeating_phase_length * explore_ratio)
    single_exploit_phase_length = repeating_phase_length - single_explore_phase_length

    single_repeating_phase = np.concatenate([
        np.ones(single_explore_phase_length),
        np.zeros(single_exploit_phase_length)
    ])

    return np.tile(single_repeating_phase, repeats)


In [ ]:
# Run the training

DESIRED_TRAIN_NUM_SECONDS = 2 * 60 * 60  # 2 hours
APPROX_EPISODES_PER_SECOND = 1100

train_num_episodes = APPROX_EPISODES_PER_SECOND * DESIRED_TRAIN_NUM_SECONDS
epsilon_fn = epsilon_picket_fence(num_episodes=train_num_episodes,
                                  explore_ratio=0.9,
                                  repeats=5)

run_training(train_from_fresh=True,
           num_episodes=train_num_episodes,
           epsilons=epsilon_fn,
           exploring_start=True)

